In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import google_streetview.api as goos

c:\users\user\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\user\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\user\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\user\appdata\local\programs\python\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
c:\users\user\appdata\local\programs\python\python37\lib\importl

Outline: 
* data sourcing
    * set up google api to download images
    * generate random lat, long
    * test location for being in urban area covered by map
    * use street view to find nearest image
    * get image metadata inc lat lon & direction
    * see if there is sidewalk centerline w/in 10ft?, create label in dataframe
* modeling
    * create tf model to classify images, train/test/validate
* use pretrained model to do object detection of sidewalks
* cluster sidewalks by type
    

In [1]:
pwd

'C:\\Users\\User\\Documents\\GitHub\\sidewalks_deep-learning'

In [8]:
#download data source if not there
from os import path
if not path.exists("sidewalks.geojson"):
    import urllib.request
    data_source_url=r'https://gis.drcog.org/geoserver/DRCOGPUB/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=DRCOGPUB:planimetrics_2016_centerline_sidewalks&outputFormat=application%2Fjson'
    urllib.request.urlretrieve(data_source_url, 'sidewalks.geojson')

In [10]:
import geopandas as gpd

In [11]:
sidewalk_gdf = gpd.read_file('sidewalks.geojson')

In [12]:
sidewalk_gdf.head()

,id,gid,createdate,createuser,type,comments,update_sta,unique_id,geometry
0,planimetrics_2016_centerline_sidewalks.534414,534414,2016-04-08,Kucera,Sidewalk,None,N,534414,"MULTILINESTRING ((3096575.759 1779104.505, 309..."
1,planimetrics_2016_centerline_sidewalks.534415,534415,2016-04-08,Kucera,Sidewalk,None,N,534415,"MULTILINESTRING ((3095429.059 1779035.067, 309..."
2,planimetrics_2016_centerline_sidewalks.534416,534416,2016-04-08,Kucera,Other crossing,None,N,534416,"MULTILINESTRING ((3096759.611 1779106.863, 309..."
3,planimetrics_2016_centerline_sidewalks.534417,534417,2016-04-08,Kucera,Sidewalk,None,N,534417,"MULTILINESTRING ((3096821.734 1779106.976, 309..."
4,planimetrics_2016_centerline_sidewalks.534418,534418,2016-04-08,Kucera,Sidewalk,None,N,534418,"MULTILINESTRING ((3096759.611 1779106.863, 309..."


In [105]:
total_bounds = sidewalk_gdf.geometry.total_bounds
total_bounds

array([2935159.7025    , 1508775.0615    , 3411162.9985    ,
       1980230.98999999])

In [114]:
#generate random locations within boundaries
import random
from shapely.geometry import Point
boundaries = sidewalk_gdf.geometry.bounds
def generate_random_loc(number, polygon=sidewalk_gdf):
    loc_list = []
    (min_x, min_y, max_x, max_y) = polygon.geometry.total_bounds
    counter = 0
    while counter < number:
        pnt = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        print(pnt)
        if polygon.geometry.contains(pnt).any(0):
            loc_list.append(pnt)
            counter += 1
    return loc_list

In [ ]:
generate_random_loc(5)

In [113]:
boundaries

,minx,miny,maxx,maxy
0,3.096501e+06,1.778880e+06,3.096591e+06,1.779105e+06
1,3.095322e+06,1.778997e+06,3.095429e+06,1.779106e+06
2,3.096760e+06,1.779107e+06,3.096822e+06,1.779107e+06
3,3.096750e+06,1.778884e+06,3.096840e+06,1.779107e+06
4,3.096576e+06,1.779105e+06,3.096760e+06,1.779108e+06
5,3.097016e+06,1.778693e+06,3.097027e+06,1.779109e+06
6,3.097016e+06,1.779109e+06,3.097065e+06,1.779109e+06
7,3.096934e+06,1.777908e+06,3.097072e+06,1.779109e+06
8,3.097777e+06,1.778422e+06,3.097787e+06,1.779110e+06
9,3.116779e+06,1.779093e+06,3.116794e+06,1.779110e+06


In [16]:
sidewalk_gdf.dtypes

id              object
gid              int64
createdate      object
createuser      object
type            object
comments        object
update_sta      object
unique_id        int64
geometry      geometry
dtype: object

In [18]:
import google_streetview.api as gsv
import os

In [36]:
#Dev key is unique to each user. To run this notebook, create your own dev key and store in directory in a text file of this name.
!google_streetview -s key="google_dev_key.key"


 Set "key" default to "google_dev_key.key"


In [49]:
import urllib, os, json
down_loc = os.getcwd()
down_loc

'C:\\Users\\User\\Documents\\GitHub\\sidewalks_deep-learning'

In [45]:
# Import google_streetview for the api module
import google_streetview 
# Import google_streetview for the api and helper module

# Create a dictionary with multiple parameters separated by ;
apiargs = {
  'location': '46.414382,10.013988;40.720032,-73.988354',
  'size': '640x300;640x640',
  'heading': '90;180',
  'fov': '90;120',
  'pitch': '0;45'
}

# Get a list of all possible queries from multiple parameters
api_list = google_streetview.helpers.api_list(apiargs)

# Create a results object for all possible queries
results = google_streetview.api.results(api_list)

# Preview results
results.preview()

# Download images to directory 'downloads'
results.download_links('downloads')

# Save metadata
results.save_metadata('metadata.json')

KeyError: 'pano_id'

In [44]:
results.links

['https://maps.googleapis.com/maps/api/streetview?location=46.414382%2C10.013988&size=640x300&heading=90&fov=90&pitch=0',
 'https://maps.googleapis.com/maps/api/streetview?location=46.414382%2C10.013988&size=640x300&heading=90&fov=90&pitch=45',
 'https://maps.googleapis.com/maps/api/streetview?location=46.414382%2C10.013988&size=640x300&heading=90&fov=120&pitch=0',
 'https://maps.googleapis.com/maps/api/streetview?location=46.414382%2C10.013988&size=640x300&heading=90&fov=120&pitch=45',
 'https://maps.googleapis.com/maps/api/streetview?location=46.414382%2C10.013988&size=640x300&heading=180&fov=90&pitch=0',
 'https://maps.googleapis.com/maps/api/streetview?location=46.414382%2C10.013988&size=640x300&heading=180&fov=90&pitch=45',
 'https://maps.googleapis.com/maps/api/streetview?location=46.414382%2C10.013988&size=640x300&heading=180&fov=120&pitch=0',
 'https://maps.googleapis.com/maps/api/streetview?location=46.414382%2C10.013988&size=640x300&heading=180&fov=120&pitch=45',
 'https://ma

In [ ]:
def MetaParse(metadata_url):
    response = urllib.urlopen(metdata_url)
    json_raw = response.read()
    json_data = json.loads(json_raw)
    #return data from json response
    if json_data['status'] == "OK":
        if 'date' in json_data:
            return (json_data['date'],json_data['pano_id']) #sometimes it does not have a date!
        else:
            return (None,json_data['pano_id'])
    else:
        return (None,None)

PrevImage = [] #Global list that has previous images sampled, memoization kindof        
        
def GetStreetvw(Lat,Lon,Head,SaveLoc):
    base = r"https://maps.googleapis.com/maps/api/streetview"
    size = r"?size=800x800&fov=90&location="
    loc = str(Lat) + "," + str(Lon) + "&heading=" + str(Head)
    file_url = base + size + loc + key
    fi = "lat" + str(Lat) + "lon" + str(Lon) + "head" + str(Head) + ".jpg"
    metadata_url = base + r"/metadata" + size + loc + key
    #print MyUrl, MetaUrl #can check out image in browser to adjust size, fov to needs
    met_lis = list(MetaParse(metadata_url))                           #does not grab image if no date
    if (met_lis[1],Head) not in PrevImage and met_lis[0] is not None:   #PrevImage is global list
        urllib.urlretrieve(file_url, os.path.join(SaveLoc,fi))
        met_lis.append(fi)
        PrevImage.append((met_lis[1],Head)) #append new Pano ID to list of images
    else:
        met_lis.append(None)
    return met_lis  


#### denver roads shapefile
roads_url = https://koordinates.com/layer/101991-denver-colorado-street-centerline/  
requires koordinates login to download

In [51]:
!unzip 'kx-denver-colorado-street-centerline-SHP.zip'

Archive:  kx-denver-colorado-street-centerline-SHP.zip
  inflating: denver-colorado-street-centerline.shx  
  inflating: denver-colorado-street-centerline.shp  
  inflating: denver-colorado-street-centerline.prj  
  inflating: denver-colorado-street-centerline.cpg  
  inflating: denver-colorado-street-centerline.dbf  
  inflating: denver-colorado-street-centerline.txt  


In [55]:
denver_streets_gdf = gpd.read_file('denver-colorado-street-centerline.shp')

In [56]:
denver_streets_gdf.head()

,FNODE,TNODE,STREETID,MASTERID,EDITDATE,L_F_ADD,L_T_ADD,R_F_ADD,R_T_ADD,PREFIX,...,T_ZLEV,ADMINCLASS,LRSROUTE,LRSCOMMENT,LRSBGN,LRSEND,HUNDBLKNAM,LRSMAJ,SHAPE_Leng,geometry
0,7833,7659,225.0,18944,20170608,561.0,599.0,560.0,598.0,N,...,0,2,ROSLYN ST7,LL,189.096668,371.004124,None,None,0.000477,"LINESTRING (-104.90097 39.72508, -104.90096 39..."
1,17472,17471,1607.0,27495,20170612,2851.0,2949.0,2850.0,2948.0,N,...,0,2,IOLA ST,LL,1003.936285,1492.316125,2850 BLK OF N IOLA ST,None,0.001341,"LINESTRING (-104.86455 39.75750, -104.86453 39..."
2,6163,6077,585.0,16501,20171024,400.0,498.0,401.0,499.0,None,...,0,2,13TH ST2,LR,142.786524,487.322780,400 BLK OF 13TH ST,None,0.001102,"LINESTRING (-104.99318 39.74039, -104.99360 39..."
3,6615,6465,402.0,4383,20020930,1301.0,1399.0,1300.0,1398.0,N,...,0,2,TENNYSON ST2,LR,458.896668,1124.667485,1300 BLK OF N TENNYSON ST,TENNYSON ST2 37,0.001828,"LINESTRING (-105.04400 39.73665, -105.04400 39..."
4,10731,10652,874.0,6514,20171024,2000.0,2098.0,2001.0,2099.0,W,...,0,2,WTENNESSEAV5,UR,1701.626782,1962.379486,2000 BLK OF W TENNESSEE AVE,None,0.000868,"LINESTRING (-105.01262 39.69832, -105.01299 39..."


In [57]:
denver_streets_gdf.plot()